In [1]:
import os
from langchain.llms import HuggingFaceHub
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone as PineconeVectorStore
from pinecone import Pinecone  # ✅ Correct import for Pinecone
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

# ✅ Load environment variables
load_dotenv()

# ✅ Hugging Face API Key (Ensure it's set in your environment variables)
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if not HUGGINGFACE_API_KEY:
    raise ValueError("❌ Missing Hugging Face API Key. Set it as HUGGINGFACEHUB_API_TOKEN")

# ✅ Load DeepSeek-R1 via API Inference
llm = HuggingFaceHub(
    repo_id="deepseek-ai/DeepSeek-R1",  # Model name
    model_kwargs={"temperature": 0.5, "max_length": 2048},
    huggingfacehub_api_token=HUGGINGFACE_API_KEY
)

# ✅ Initialize Pinecone Client
pinecone_client = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# ✅ Ensure the new Pinecone index name is set
index_name = os.getenv("PINECONE_INDEX_V2")  # ✅ Updated to use the latest index
if not index_name:
    raise ValueError("❌ PINECONE_INDEX_V2 is not set. Check your .env file.")

# ✅ Ensure the index exists before using it
existing_indexes = [idx["name"] for idx in pinecone_client.list_indexes()]
if index_name not in existing_indexes:
    raise ValueError(f"❌ Index '{index_name}' does not exist in Pinecone. Please create it first.")

# ✅ Initialize Pinecone Index
index = pinecone_client.Index(index_name)

# ✅ Initialize the same embedding model used for storing embeddings
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# ✅ Load the vector store for retrieval
vectorstore = PineconeVectorStore(
    index=index,  # ✅ Connects to the updated Pinecone index
    embedding=huggingface_embeddings,
    text_key="text"  # ✅ Specifies where text metadata is stored
)

# ✅ Create a retriever from the vector store
retriever = vectorstore.as_retriever(
    search_type="similarity",  
    search_kwargs={"k": 3}  # Retrieve top 3 most relevant chunks
)

# ✅ Define the Prompt Template for RAG-based Q&A
prompt_template = """
You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. If the query references a specific call (e.g., "call_004"), only retrieve information from the corresponding "call_id".
4. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: {context}
</ctx>
-------
<hs>
Chat History: {chat_history}
</hs>
-------
Question: {question}

Useful Answer:
"""


PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "chat_history", "question"]  # ✅ Includes chat history now
)


# ✅ Memory buffer to handle multiple interactions
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="question"
)

# ✅ Define Retrieval-Based Q&A Chain
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT, "memory": memory}
)

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
C:\Users\Claudio\AppData\Local\Temp\ipykernel_30472\2620012479.py:20: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(
C:\Users\Claudio\AppData\Local\Temp\ipykernel_30472\2620012479.py:43: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingf

In [11]:
memory.clear() # para resetear memoria (si no se hace, se puede superar el limite de tokens y da error)

In [2]:
query = "What was the remaining balance in call_004?"
#query = "What is the agent name in `call_001`?"
#query = "What is the balance in 'call_002'?"
response = retrievalQA.invoke({"query": query})

print(response["result"])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. If the query references a specific call (e.g., "call_004"), only retrieve information from the corresponding "call_id".
4. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: Agent: Good morning, this is Alex from Debt Relief Services. I’m contacting you about a $200 balance.
Client: I made a payment last week.
Agent: Let me confirm… Yes, we received it. Your remaining balance is $50.
Client: I’ll clear that today.
Agent: Would you like to pay over the phone?
Client: Yes, let’s do that.

Agent: Hello, this is Sara

In [3]:
query = "Which calls mention the word 'extensions'?"
#query = "Which calls mention 'payment plan' and which are the ID of these calls?"
#query = "Which calls mention the word 'confirmation'?"
response = retrievalQA.invoke({"query": query})

print(response['result'])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. If the query references a specific call (e.g., "call_004"), only retrieve information from the corresponding "call_id".
4. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: Agent: Hi, this is Tom from Billing Support. I see a new charge of $600.
Client: What is that for?
Agent: It’s a late fee and interest on a past-due balance.
Client: But I had an extension!
Agent: It expired last month.
Client: Can you remove the late fee?
Agent: I’ll check and get back to you.

Client: That’s better. When do I have to start?

In [4]:
query = "is there any call that talks about loosing a job? give me the specific ID call."
response = retrievalQA.invoke({"query": query})

print(response['result'])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. If the query references a specific call (e.g., "call_004"), only retrieve information from the corresponding "call_id".
4. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: Agent: Good afternoon, this is John from Recovery Services. I see your balance of $350 is overdue.
Client: Yes, I just lost my job and can’t make a payment right now.
Agent: I’m very sorry to hear that. Would a one-month extension help?
Client: It would, but I don’t know if I’ll have a job by then.

Agent: Good afternoon, this is Amy from Fin

In [5]:
#query = "give me a summary from the call_004"
query = "give me a summary from the call_003"

response = retrievalQA.invoke({"query": query})

print(response['result'])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. If the query references a specific call (e.g., "call_004"), only retrieve information from the corresponding "call_id".
4. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: Agent: Good morning, this is Alex from Debt Relief Services. I’m contacting you about a $200 balance.
Client: I made a payment last week.
Agent: Let me confirm… Yes, we received it. Your remaining balance is $50.
Client: I’ll clear that today.
Agent: Would you like to pay over the phone?
Client: Yes, let’s do that.

Agent: Hello, this is Sara

In [7]:
query = "what call did John make?"

response = retrievalQA.invoke({"query": query})

print(response['result'])

c:\Users\Claudio\tfm_call_optimizer\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



You are an expert auditor analyzing call center transcription calls from custumer support calls.
You need to give useful insights from the questions an audit expert would make so he can understand how the calls went.

Instructions:
1. Answer based on the provided context (delimited by <ctx> </ctx>) and the chat history (delimited by <hs> </hs>) below.
2. If the information is not in the context, respond: "I don't have this information."
3. If the query references a specific call (e.g., "call_004"), only retrieve information from the corresponding "call_id".
4. **Provide a concise and precise answer.** 

Provided Information
-------
<ctx>
Context: Agent: Good morning, this is Alex from Debt Relief Services. I’m contacting you about a $200 balance.
Client: I made a payment last week.
Agent: Let me confirm… Yes, we received it. Your remaining balance is $50.
Client: I’ll clear that today.
Agent: Would you like to pay over the phone?
Client: Yes, let’s do that.

Agent: Would you like to p